In [0]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

E: Package 'python-software-properties' has no installation candidate
··········


In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive
import os
os.chdir("drive/Colab Notebooks") 
!ls

fuse: mountpoint is not empty
fuse: if you are sure this is safe, use the 'nonempty' mount option
 1-1_tensor_tutorial.ipynb		   RNN_project.ipynb
 1-2_autograd_tutorial.ipynb		   RNN_project_slidingW.ipynb
'1-3_neural_networks_tutorial (1).ipynb'  'Un (a5fafc8c).ipynb'
'1-4_cifar10_tutorial (1).ipynb'	   Untitled
 1_hello_tensorflow.ipynb		   Untitled0.ipynb
 2_getting_started.ipynb		   Untitled1.ipynb
'3_mnist_from_scratch (6f53deeb).ipynb'    Untitled2.ipynb
 3_mnist_from_scratch.ipynb		   Untitled3.ipynb
 data					   Untitled4.ipynb
 ee239_1000epoch.ipynb			   Untitled5.ipynb
 HW5_CNN.ipynb				   Untitled6.ipynb
 results				   Untitled7.ipynb
'RNN_project (58c845cd).ipynb'


In [0]:
import os

import torch
import torchvision
from torch import nn
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.utils import save_image

import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets

import numpy as np
import matplotlib.pyplot as plt







X_test = np.load("data/X_test.npy")
y_test = np.load("data/y_test.npy")
person_train_valid = np.load("data/person_train_valid.npy")
X_train_valid = np.load("data/X_train_valid.npy")
y_train_valid = np.load("data/y_train_valid.npy")
person_test = np.load("data/person_test.npy")

print ('Training/Valid data shape: {}'.format(X_train_valid.shape))
print ('Test data shape: {}'.format(X_test.shape))
print ('Training/Valid target shape: {}'.format(y_train_valid.shape))
print ('Test target shape: {}'.format(y_test.shape))
print ('Person train/valid shape: {}'.format(person_train_valid.shape))
print ('Person test shape: {}'.format(person_test.shape))

Training/Valid data shape: (2115, 25, 1000)
Test data shape: (443, 25, 1000)
Training/Valid target shape: (2115,)
Test target shape: (443,)
Person train/valid shape: (2115, 1)
Person test shape: (443, 1)


In [0]:
# import torch 
# import torchvision.datasets as dsets
# import torchvision.transforms as transforms
# import torchvision
# from torch.autograd import Variable

# from time import time

# # from AE import *







In [0]:
from time import time


In [0]:
batch_size = 50
num_epoches = 50

window_step = 5
window_length = 10

learning_rate = 1e-3
betas=(0.9, 0.999)
weight_decay=1e-3

In [0]:

def flatten(x):
    """
    Input:
    - Tensor of shape (N, D1, ..., DM)
    Output:
    - Tensor of shape (N, D1 * ... * DM)
    """
    x_shape = x.size()
    new_shape = 1
    for i in range(len(x_shape) - 1):
        new_shape *= x_shape[i + 1]
    x_flat = x.reshape((x_shape[0],new_shape))
    return x_flat
  
def sliding_window(length, shift, data):
    """
    Input:
    - raw eeg data
    Output:
    - processed window signals, everytime shifted with "shift" units, window length is "length"
    """
    p, c, e, d = data.shape
    
    if(d%shift==0):
        new_data_size = d//shift
        new_data = np.zeros((p, c, e, new_data_size))
        for i in range(d//shift):
            new_data[:, :, :, i] = np.mean(data[:, :, :, i*shift:(i+1)*shift+length], axis = 3)

    else:
        new_data_size = d//shift+1
        new_data = np.zeros((p, c, e, new_data_size))
        for i in range(d//shift):
            new_data[:, :, :, i] = np.mean(data[:, :, :, i*shift:(i+1)*shift+length], axis = 3)
        new_data[:, :, :, new_data_size-1] = np.mean(data[:, :, :, (new_data_size-1)*shift:], axis = 3)
    
    return new_data

In [0]:
X_train = sliding_window(window_length, window_step, X_train_valid.reshape((2115, 1, 25, 1000)))
X_test = sliding_window(window_length, window_step, X_test.reshape((443, 1, 25, 1000)))


trainset = torch.utils.data.TensorDataset(torch.from_numpy(X_train[:, :, :22, :]).float(), torch.from_numpy(y_train_valid - 769).long())
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=2)

In [0]:
print (format(X_train.shape))

(2115, 1, 25, 200)


In [0]:
def to_var(x):
    if torch.cuda.is_available():
        x = x.cuda()
    return Variable(x)


class Autoencoder(nn.Module):
    def __init__(self, in_dim=22*200, h_dim=11*200):
        super(Autoencoder, self).__init__()

        self.encoder = nn.Sequential(
            nn.Linear(22*200, 14*200),
            nn.ReLU(),
            nn.Linear(14*200, 11*200),
            nn.ReLU()
            )

        self.decoder = nn.Sequential(
            nn.Linear(11*200, 14*200),
            nn.ReLU(),
            nn.Linear(14*200, 22*200),
            nn.Sigmoid()
            )


    def forward(self, x):
        """
        Note: image dimension conversion will be handled by external methods
        """
        out = self.encoder(x)
        out = self.decoder(out)
        return out


In [0]:
ae = Autoencoder()

if torch.cuda.is_available():
    ae.cuda()

criterion = nn.MSELoss()
optimizer = torch.optim.Adam(ae.parameters(), lr=learning_rate, weight_decay=weight_decay)
iter_per_epoch = len(trainloader)
data_iter = iter(trainloader)

# # save fixed inputs for debugging
# fixed_x, _ = next(data_iter)
# fixed_x = to_var(fixed_x.view(fixed_x.size(0), -1))

for epoch in range(num_epochs):
    t0 = time()
    for i, (images, _) in enumerate(trainloader):

        # flatten the image
        images = to_var(images.view(images.size(0), -1))
        out = ae(images)
        loss = criterion(out, images)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 1 == 0:
            print ('Epoch [%d/%d], Iter [%d/%d] Loss: %.4f Time: %.2fs' 
                %(epoch+1, num_epochs, i+1, len(trainset)//batch_size, loss.data.item(), time()-t0))

    # save the reconstructed images


Epoch [1/50], Iter [1/42] Loss: 58.0819 Time: 0.17s
Epoch [1/50], Iter [2/42] Loss: 50.4032 Time: 0.22s
Epoch [1/50], Iter [3/42] Loss: 54.2593 Time: 0.27s
Epoch [1/50], Iter [4/42] Loss: 54.9226 Time: 0.31s
Epoch [1/50], Iter [5/42] Loss: 55.6743 Time: 0.35s
Epoch [1/50], Iter [6/42] Loss: 59.9396 Time: 0.40s
Epoch [1/50], Iter [7/42] Loss: 56.2857 Time: 0.44s
Epoch [1/50], Iter [8/42] Loss: 53.4827 Time: 0.48s
Epoch [1/50], Iter [9/42] Loss: 56.4554 Time: 0.52s
Epoch [1/50], Iter [10/42] Loss: 55.7063 Time: 0.56s
Epoch [1/50], Iter [11/42] Loss: 54.8033 Time: 0.61s
Epoch [1/50], Iter [12/42] Loss: 63.4960 Time: 0.65s
Epoch [1/50], Iter [13/42] Loss: 58.2810 Time: 0.68s
Epoch [1/50], Iter [14/42] Loss: 58.0615 Time: 0.72s
Epoch [1/50], Iter [15/42] Loss: 53.5478 Time: 0.76s
Epoch [1/50], Iter [16/42] Loss: 52.2975 Time: 0.80s
Epoch [1/50], Iter [17/42] Loss: 60.8008 Time: 0.84s
Epoch [1/50], Iter [18/42] Loss: 58.7472 Time: 0.87s
Epoch [1/50], Iter [19/42] Loss: 46.6299 Time: 0.91s
Ep

In [0]:
print(iter_per_epoch)

43
